In [1]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv
import pickle
import numpy as np
import pandas as pd
import pickle

from multiprocessing import Process
from multiprocessing import Pool
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

In [2]:

options = Options()

options.add_argument('--user-data-dir=/Users/Arian_Gh/AppData/Local/Google/Chrome/User Data/') 
options.add_argument('--profile-directory=Profile 6')
options.add_argument("--disable-extensions")
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--start-maximized")

options.add_argument('disable-infobars')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--remote-debugging-port=9222")
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
options.add_argument(f'user-agent={user_agent}')

prefs = {
    "profile.default_content_setting_values.images": 2
}
options.add_experimental_option("prefs", prefs)

# Add a preference to disable image loading


capabilities = DesiredCapabilities.CHROME.copy()
capabilities['acceptSslCerts'] = True
capabilities['acceptInsecureCerts'] = True




In [3]:
driver = webdriver.Chrome(options=options , desired_capabilities=capabilities)
driver.get("https://www.plastic4trade.com/")

In [17]:
with open("CategoryUrls","rb") as f:
    category_urls = pickle.load(f)

In [19]:
driver.get(category_urls[0])

In [20]:
sale_btn = driver.find_element(By.XPATH,"//*[@id='salepostcategory']/div/a/button")
buy_btn = driver.find_element(By.XPATH,"//*[@id='buypostcategory']/div/a/button")

In [21]:
sale_btn.click()

In [24]:
html_text = driver.page_source
soup = BeautifulSoup(html_text,"lxml")

In [43]:
for category_url in category_urls:
    products_part = soup.find("div",class_="news-detail-right-main")
    products = products_part.find_all("div",class_="col-lg-3 col-md-3 col-sm-12 col-xs-12")
    details = {}
    for product in products:
        details["name"] = product.find("h6").text
        details["link"] = product.find("a")["href"]
        details["price"] = product.find_all("div")[-1].text.strip()
        details["category"] = category_url.split("/")[-2]
        details["type"] = "sell" if is_sell else "buy"